In [22]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

In [57]:
# Load in array of 201 images, create dataframe
phi = np.load("Phi_201snap.npy")
df = pd.DataFrame(index=np.arange(201), columns=np.arange(1))
df.rename(columns={0: 'norm'}, inplace=True)

r = np.linspace(1, 101, 101)
theta = np.linspace(0, 2 * np.pi, 227)
r_grid, theta_grid = np.meshgrid(r, theta, indexing='ij')

# Convert polar coordinates to Cartesian coordinates
x = r_grid * np.cos(theta_grid)
y = r_grid * np.sin(theta_grid)

# Define the target grid for interpolation (203x203 pixels)
grid_x, grid_y = np.linspace(-101, 101, 203), np.linspace(-101, 101, 203)
grid_x, grid_y = np.meshgrid(grid_x, grid_y)

points = np.array([x.flatten(), y.flatten()]).T

for i in range(201):
    data = phi[:,:,i]
    values = data.flatten()
    
    # Interpolate the data to the Cartesian grid
    grid_z = griddata(points, values, (grid_x, grid_y), method='nearest')
    
    # Optional: Plot and save image
    # plt.imshow(grid_z, extent=(-101, 101, -101, 101), cmap='gray')
    # plt.axis("off")
    # plt.savefig('image.jpg', bbox_inches='tight')
    # plt.close()

    output_array = np.zeros((101, 227))
    for rad in range(101):
        for ang in range(227):
            # Polar coordinates
            radius = r[rad]
            angle = theta[ang]
            
            # Convert to Cartesian coordinates
            x_cart = radius * np.cos(angle)
            y_cart = radius * np.sin(angle)
            
            # Find the nearest pixel in the Cartesian grid
            x_nearest = np.argmin(np.abs(grid_x[0] - x_cart))
            y_nearest = np.argmin(np.abs(grid_y[:, 0] - y_cart))
            
            # Get the grayscale value from the Cartesian image
            output_array[rad, ang] = grid_z[y_nearest, x_nearest]
            
    # Compute norm, add to dataframe
    norm_sq = 0
    diff_array = output_array - data
    norm = np.linalg.norm(diff_array)

    # for row in range(101):
    #     for col in range(227):
    #         norm_sq += (diff_array[row][col])**2
    # norm = norm_sq**0.5
    
    df.at[i, 'norm'] = norm

In [58]:
df

,norm
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
196,0.48222
197,0.520447
198,0.572255
199,0.641622
